In [1]:
import pandas as pd
import numpy as np
import os
from os.path import join
from tqdm import tqdm
import re

In [24]:
#### 재작성할 증서목록 파일 ####
wd = r"D:\3.자산\실물증서정리"
filename = "list.xlsx"
##############################
path_excel = join(wd, filename)
ori_df = pd.read_excel(path_excel, dtype=str)
ori_df

,정렬용순번,원장번호,채무자키
0,DNP-0001,1,20411331
1,DNP-0002,2,20411332
2,DNP-0003,3,20416214
3,DNP-0004,4,20457194
4,DNP-0005,5,20416220
...,...,...,...
8585,한울-4346,4208,20413979
8586,한울-4347,4209,20413322
8587,한울-4348,4210,20413940
8588,한울-4349,4211,20411164


In [1]:
# 쉼표, /, () 사용한 셀 있는지 체크하고 수작업하거나, 그에 맞춰 코드 수정

# 문자열 오름차순 정렬 순서 : 숫자형태만 쓰면 상관없으나 기호가 섞이면 문제가 됨.
# 기준1) 숫자문자열은 zfill한 것처럼 정렬됨 >>> 1, 2, 10으로 문제없음. 그러나 언더바나 하이픈 같은 기호 다음은 zfill 적용x >>> 1_1, 1_10, 1_2
# 기준2) 숫자문자열과 기호 : 0, -, _ >>> 따라서 10이 1_1보다 먼저 나와버림
# 결론1) -, _를 쓰건 안 쓰건 분철번호는 자릿수를 맞춰서 해야함 (순번이 필요 없지만 가독성이 안 좋은 단점)
# 결론2) -나 _를 쓰려면 전체에 똑같이 사용해줘야 함. (아래 코드는 이걸 새순번으로 만듦)

In [25]:
# 전처리 : 에러 확인
물결 = re.compile("~")
하이픈 = re.compile("\-")
for i, v in tqdm(ori_df.iterrows(), total=len(ori_df.index)) :
    if 물결.search(v.원장번호) : 
        start = 물결.search(v.원장번호).start()
        end = 물결.search(v.원장번호).end()
        앞번호 = v.원장번호[:start]
        뒷번호 = v.원장번호[end:]
        # 물결 2개 이상
        if len(물결.findall(v.원장번호)) > 1 : 
            print("물결 2개 이상", v.원장번호)
        # 하이픈 2개이상
        if len(하이픈.findall(앞번호))>1 or len(하이픈.findall(뒷번호))>1 :
            print("하이픈 2개 이상", v.원장번호)
        
    else : 
        if len(하이픈.findall(v.원장번호)) > 1 : 
            start = 물결.search(v.원장번호).start()
            end = 물결.search(v.원장번호).end()
            앞번호 = v.원장번호[:start]
            뒷번호 = v.원장번호[end:]
            # 하이픈 2개이상
            if len(하이픈.findall(앞번호))>1 or len(하이픈.findall(뒷번호))>1 :
                print("하이픈 2개 이상", v.원장번호)
                
# (부연설명) 지우기 : 여기서 지우면 부연설명을 더는 볼수가 없네..
# ori_df["원장번호"] = ori_df["원장번호"].apply(lambda x : 부연설명.sub("", x))

100%|██████████| 8590/8590 [00:00<00:00, 36970.34it/s]


In [27]:
정렬용순번, 매각사, 원장번호, 새순번, 새증서번호, 채무자키, 비고 = [],[],[],[],[], [], []
물결 = re.compile("~")
하이픈 = re.compile("-")
부연설명 = re.compile("\(.*\)")
for i, v in tqdm(ori_df.iterrows(), total=len(ori_df.index)) :
    try : 
        원래원장번호 = 부연설명.sub("", v.원장번호)    
        
        # 물결 있으면
        if 물결.search(원래원장번호) : 
            start = 물결.search(원래원장번호).start()
            end = 물결.search(원래원장번호).end()
            앞번호 = 원래원장번호[:start]
            뒷번호 = 원래원장번호[end:]
            
            # 앞번호
            # 앞번호에 하이픈 있으면
            if 하이픈.search(앞번호) :
                # 4자리 증서번호 만들기
                앞증서번호 = 앞번호[ : 하이픈.search(앞번호).start()]
                # 2자리 분철번호 만들기 (int)
                앞분철번호 = 앞번호[하이픈.search(앞번호).end() : ]
            # 앞번호에 하이픈 없으면
            else : 
                앞증서번호 = 앞번호
                앞분철번호 = "0"
            
            # 뒷번호
            if 하이픈.search(뒷번호) :
                뒷증서번호 = 뒷번호[ : 하이픈.search(뒷번호).start()]
                뒷분철번호 = 뒷번호[하이픈.search(뒷번호).end() : ]
            else : 
                뒷증서번호 = 뒷번호
                뒷분철번호 = "0"

            # 증서번호 같은가?
            if 앞증서번호 == 뒷증서번호 : 
                증서번호 = 앞증서번호
                # 분철번호만큼 반복해서 값 채우기
                for i in range(int(앞분철번호), int(뒷분철번호)+1) : ############값채우기
                    분철번호 = str(i)
                    정렬용순번.append(v.정렬용순번)
                    매각사.append(v.정렬용순번[:하이픈.search(v.정렬용순번).start()])
                    원장번호.append(v.원장번호)
                    새순번.append(증서번호.zfill(4) + "_" + 분철번호.zfill(2))
                    새증서번호.append(증서번호 if 분철번호 == "0" else 증서번호 + "_" + 분철번호)
                    채무자키.append(v.채무자키)
                    비고.append("")
            # error : 물결있는데 증서번호가 다른 경우
            else : 
                raise Exception("앞증서번호와 뒷증서번호가 다름")
        
        # 물결 없으면
        else :
            # 하이픈 있으면
            if 하이픈.search(원래원장번호) : 
                증서번호 = 원래원장번호[ : 하이픈.search(원래원장번호).start()]
                분철번호 = 원래원장번호[하이픈.search(원래원장번호).end():]
            # 하이픈 없으면
            else : 
                증서번호 = 원래원장번호
                분철번호 = "0"

            정렬용순번.append(v.정렬용순번) ############값채우기
            매각사.append(v.정렬용순번[:하이픈.search(v.정렬용순번).start()])
            원장번호.append(v.원장번호)
            새순번.append(증서번호.zfill(4) + "_" + 분철번호.zfill(2))
            새증서번호.append(증서번호 if 분철번호 == "0" else 증서번호 + "_" + 분철번호)
            채무자키.append(v.채무자키)
            비고.append("")
    
    except Exception as e :
        정렬용순번.append(v.정렬용순번)
        매각사.append(v.정렬용순번[:하이픈.search(v.정렬용순번).start()])
        원장번호.append(원래원장번호)
        새순번.append("") ###
        새증서번호.append("") ###
        채무자키.append(v.채무자키)
        비고.append(e.args[0])
        continue

100%|██████████| 8590/8590 [00:00<00:00, 22621.35it/s]


In [23]:
# 결과 저장하기
result_df = pd.DataFrame({
    "정렬용순번":정렬용순번,
    "매각사":매각사,
    "원장번호":원장번호,
    "새순번":새순번,
    "새증서번호":새증서번호,
    "채무자키":채무자키,
    "에러":에러
})
result_df.to_excel(join(wd, "result_list.xlsx"), index=False)

# 분철행 추가하기

In [15]:
#### 재작성할 증서목록 파일 ####
wd = r"D:\3.자산\실물증서정리"
filename = "0.광주 증서목록 정리본.xlsx"
##############################
path_excel = join(wd, filename)
ori_df = pd.read_excel(path_excel, sheet_name="리스트", dtype=str).fillna("")
# 정렬번호로 컬럼명 바꾸기
# ori_df.rename(columns={"정렬(not unique)":"정렬번호"}, inplace=True)
# 정렬
ori_df.sort_values("정렬번호", inplace=True)
ori_df

,구정렬용순번,증서상 매각사,실제매각사,원증서번호,정렬용,정렬번호,새증서번호,채무자키,채무자명,채무자주민번호,에러,에이원,동양,계좌분리건
0,DNP-0001,DNP,,1,0001_00,DNP0001_00,1,20411331,김정욱,570827-1,,0,0,0
1,DNP-0002,DNP,,2,0002_00,DNP0002_00,2,20411332,조봉연,570828-2,,0,0,0
2,DNP-0003,DNP,,3,0003_00,DNP0003_00,3,20416214,강신석,570906-1,,0,0,0
3,DNP-0004,DNP,,4,0004_00,DNP0004_00,4,20457194,안태순,570910-2,,0,0,0
4,DNP-0005,DNP,,5,0005_00,DNP0005_00,5,20416220,최선녀,570923-2,,0,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8590,한울-4346,한울,,4208,4208_00,한울4208_00,4208,20413979,홍정민(홍영표 변경전),580623-1,,0,0,0
8591,한울-4347,한울,,4209,4209_00,한울4209_00,4209,20413322,이미대자,410607-2,,0,0,0
8592,한울-4348,한울,,4210,4210_00,한울4210_00,4210,20413940,노순성,650510-1,,0,0,0
8593,한울-4349,한울,,4211,4211_00,한울4211_00,4211,20411164,김정옥,610420-2,,0,0,0


In [17]:
# 대표번호, 분철번호열 추가하기
ori_df["대표번호"] = ori_df.정렬번호.str.split('_').str[0]
ori_df["분철번호"] = ori_df.정렬번호.str.split('_').str[1]
ori_df.head()

,구정렬용순번,증서상 매각사,실제매각사,원증서번호,정렬용,정렬번호,새증서번호,채무자키,채무자명,채무자주민번호,에러,에이원,동양,계좌분리건,대표번호,분철번호
0,DNP-0001,DNP,,1,0001_00,DNP0001_00,1,20411331,김정욱,570827-1,,0,0,0,DNP0001,00
1,DNP-0002,DNP,,2,0002_00,DNP0002_00,2,20411332,조봉연,570828-2,,0,0,0,DNP0002,00
2,DNP-0003,DNP,,3,0003_00,DNP0003_00,3,20416214,강신석,570906-1,,0,0,0,DNP0003,00
3,DNP-0004,DNP,,4,0004_00,DNP0004_00,4,20457194,안태순,570910-2,,0,0,0,DNP0004,00
4,DNP-0005,DNP,,5,0005_00,DNP0005_00,5,20416220,최선녀,570923-2,,0,3,0,DNP0005,00


In [40]:
# 대표번호로 그룹화하기
grouped_df = ori_df.groupby("대표번호")

In [60]:
ori_df.columns

Index(['구정렬용순번', '증서상 매각사', '실제매각사', '원증서번호', '정렬용', '정렬번호', '새증서번호', '채무자키',
       '채무자명', '채무자주민번호', '에러', '에이원', '동양', '계좌분리건', '대표번호', '분철번호'],
      dtype='object')

In [73]:
result_df = pd.DataFrame(None, columns=ori_df.columns)
for g_name, g_data in tqdm(grouped_df, total=len(grouped_df)):
    result_df = pd.concat([result_df, g_data])
    분철번호 = g_data["분철번호"].astype(int)
    최소값 = 분철번호.min()
    최대값 = 분철번호.max()

    # 연속되지 않는 분철번호 찾기
    missing = [num for num in range(최소값, 최대값+1) if num not in 분철번호.values]

    if missing :
        temp_df = pd.DataFrame([{}]*len(missing), columns=ori_df.columns)
        temp_df["분철번호"] = [str(i).zfill(2) for i in missing]
        temp_df["정렬용"] = [g_data["정렬용"].values[0][:-2] + x for x in temp_df["분철번호"].values]
        temp_df["정렬번호"] = [g_data["정렬번호"].values[0][:-2] + x for x in temp_df["분철번호"].values]
        temp_df["대표번호"] = g_data["정렬번호"].values[0][:-3]
        temp_df[["구정렬용순번", "원증서번호", "실제매각사", "원증서번호", "새증서번호", "에러", "에이원", "동양", "계좌분리건"]]= ""
        temp_df[["증서상 매각사", "채무자키", "채무자명", "채무자주민번호"]] = g_data.loc[g_data.index[0],["증서상 매각사", "채무자키", "채무자명", "채무자주민번호"]]
        result_df = pd.concat([result_df, temp_df])

result_df

100%|██████████| 8104/8104 [00:06<00:00, 1178.83it/s]


,구정렬용순번,증서상 매각사,실제매각사,원증서번호,정렬용,정렬번호,새증서번호,채무자키,채무자명,채무자주민번호,에러,에이원,동양,계좌분리건,대표번호,분철번호
0,DNP-0001,DNP,,1,0001_00,DNP0001_00,1,20411331,김정욱,570827-1,,0,0,0,DNP0001,00
1,DNP-0002,DNP,,2,0002_00,DNP0002_00,2,20411332,조봉연,570828-2,,0,0,0,DNP0002,00
2,DNP-0003,DNP,,3,0003_00,DNP0003_00,3,20416214,강신석,570906-1,,0,0,0,DNP0003,00
3,DNP-0004,DNP,,4,0004_00,DNP0004_00,4,20457194,안태순,570910-2,,0,0,0,DNP0004,00
4,DNP-0005,DNP,,5,0005_00,DNP0005_00,5,20416220,최선녀,570923-2,,0,3,0,DNP0005,00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8590,한울-4346,한울,,4208,4208_00,한울4208_00,4208,20413979,홍정민(홍영표 변경전),580623-1,,0,0,0,한울4208,00
8591,한울-4347,한울,,4209,4209_00,한울4209_00,4209,20413322,이미대자,410607-2,,0,0,0,한울4209,00
8592,한울-4348,한울,,4210,4210_00,한울4210_00,4210,20413940,노순성,650510-1,,0,0,0,한울4210,00
8593,한울-4349,한울,,4211,4211_00,한울4211_00,4211,20411164,김정옥,610420-2,,0,0,0,한울4211,00


In [74]:
result_df.to_excel(join(wd, "분철번호채우기.xlsx"), index=False)

In [43]:
# 그룹화된 DataFrame을 리스트로 저장
grouped_dataframes = [g_data for g_name, g_data in grouped_df]

# 그룹화된 데이터프레임을 일반 데이터프레임으로 변환
merged_df = pd.concat(grouped_dataframes)

# 데이터프레임을 엑셀 파일로 저장
merged_df.to_excel('xxx.xlsx', index=False)